In [1]:
import pandas as pd
import joblib
import time

from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV  # Bayesian optimization: utilizado para optimizar hiperparámetros

import lightgbm as lgbm
from lightgbm import early_stopping  # Early stopping: utilizado para evitar sobreajuste

from Funcoes_Comuns import avaliar_modelo, registrar_modelo

In [2]:
# Obter dados
df_enem = pd.read_pickle('Bases\microdados_enem_censo_2023.pkl')

In [3]:
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Ajuste de tipo para MLflow
# Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

In [5]:
categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

categorical_features

['CAT_COR_RACA',
 'CAT_CO_UF_ESC',
 'CAT_ENSINO',
 'CAT_ESCOLA',
 'CAT_ESTADO_CIVIL',
 'CAT_FAIXA_ETARIA',
 'CAT_LINGUA',
 'CAT_NACIONALIDADE',
 'CAT_Q003',
 'CAT_Q004',
 'CAT_SEXO',
 'CAT_SIT_FUNC_ESC',
 'CAT_MODE_CATEGORIA_ESCOLA_PRIVADA',
 'CAT_MODE_EXAME_SELECAO',
 'CAT_MODE_LOCALIZACAO_DIFERENCIADA',
 'CAT_MODE_OCUPACAO_GALPAO',
 'CAT_MODE_OCUPACAO_PREDIO_ESCOLAR',
 'CAT_MODE_ORGAO_REGIONAL',
 'CAT_MODE_PROPOSTA_PEDAGOGICA',
 'CAT_MODE_REGIAO',
 'CAT_MODE_TRATAMENTO_LIXO_INEXISTENTE']

Modelo base

In [6]:
# Treinar modelo LGBMRegressor Base
modelo_lgbm = lgbm.LGBMRegressor(n_estimators=1000, 
                                 learning_rate=0.01, 
                                 random_state=42,
                                 max_bin=4095,
                                 force_row_wise=True)

start_time = time.time()

modelo_lgbm.fit(X_train, 
                y_train['NUM_NOTA_CH'], 
                eval_set=[(X_test, y_test['NUM_NOTA_CH'])], 
                eval_metric=['r2', 'rmse', 'mae'],
                categorical_feature=categorical_features)

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 29930
[LightGBM] [Info] Number of data points in the train set: 573194, number of used features: 106
[LightGBM] [Info] Start training from score 527.941622


In [7]:
# Previsões
y_pred = modelo_lgbm.predict(X_test)

In [8]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                 parametros={**modelo_lgbm.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                 X_train=X_train,
                 y_train=y_train,
                 y_test=y_test,
                 y_pred=y_pred,
                 variavel_alvo='NUM_NOTA_CH',
                 modelo=modelo_lgbm,
                 nome_modelo='modelo_lgbm_base_censo_enem',
                 descricao_modelo='Modelo LGBMRegressor base Censo e ENEM 2023')

2025/06/04 21:05:34 WARNING mlflow.models.signature: Failed to infer schema for inputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. Note that MLflow doesn't validate data types during inference for AnyType. To see the full traceback, set logging level to DEBUG.
Successfully registered model 'modelo_lgbm_base_censo_enem'.
2025/06/04 21:06:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_lgbm_base_censo_enem, version 1


🏃 View run brawny-sloth-372 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/f013ab98f0f848cf89f15b5ac088cd36
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_base_censo_enem
Rastreamento do MLflow finalizado.


Created version '1' of model 'modelo_lgbm_base_censo_enem'.


In [9]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred, "teste")

MAE (treino): 54.7139
RMSE (treino): 69.3612
R2 (treino): 0.3287
MAE (teste): 55.1720
RMSE (teste): 69.8875
R2 (teste): 0.3149


Bayes Search

In [17]:
modelo_lgbm_bayes = lgbm.LGBMRegressor(random_state=42, 
                                       early_stopping_rounds=200,
                                       max_bin=4095, 
                                       force_row_wise=True)

In [18]:
# Definição do espaço de busca para otimização bayesiana
param_grid = {
    'num_leaves': (5, 60),                         # Número de folhas na árvore de decisão
    'max_depth': (40, 100),                        # Profundidade máxima da árvore
    'learning_rate': (0.005, 0.1, 'log-uniform'),  # Taxa de aprendizado
    'n_estimators': (5000, 6000),                  # Número de árvores
    'subsample': (0.3, 1.0),                       # Proporção de amostras usadas em cada árvore
    'colsample_bytree': (0.2, 1.0),                # Fração de colunas a serem usadas por árvore
    'reg_alpha': (1e-3, 1.0, 'log-uniform'),       # Regularização L1
    'reg_lambda': (1e-5, 1.0, 'log-uniform'),      # Regularização L2
}

In [19]:
# Configurar a busca Bayesiana usando BayesSearchCV

# Criando o otimizador Bayesiano
bayes_search = BayesSearchCV(
    estimator=modelo_lgbm_bayes,    # Modelo a ser otimizado
    search_spaces=param_grid,       # Espaço de busca definido acima
    scoring='r2',                   # Critério de seleção
    n_iter=30,                      # Número de avaliações do modelo
    cv=5,                           # Validação cruzada
    random_state=42,                # Semente para reprodutibilidade
    n_jobs=-1,                      # Paralelização total dos cálculos
    verbose=1                       # 0 = sem mensagens, 1 = mensagens de progresso, 2 = mensagens detalhadas
)

In [20]:
# Criar Eval Set para validação cruzada (15% do conjunto de treino)
X_train_bayes, X_eval, y_train_bayes, y_eval = train_test_split(
    X_train,
    y_train['NUM_NOTA_CH'],
    test_size=0.15,
    random_state=42
)

In [21]:
fit_params = {
    'eval_set': [(X_eval, y_eval)],               # Conjunto de validação
    'eval_metric': ['r2', 'rmse', 'mae'],         # Métricas a serem avaliadas
    'categorical_feature': categorical_features,  # Colunas categóricas
}

In [22]:
# Executar a busca Bayesiana

start_time = time.time()
bayes_search.fit(X_train_bayes, y_train_bayes, **fit_params)

# Parar o cronômetro
end_time = time.time()
elapsed_time = end_time - start_time

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

In [23]:
# Resultados da busca Bayesiana

print("Melhores parâmetros encontrados:")
print(bayes_search.best_params_)
print("R2: ", bayes_search.best_score_)
print(f"Tempo total de execução: {elapsed_time:.2f} segundos")

Melhores parâmetros encontrados:
OrderedDict([('colsample_bytree', 0.2663042735572549), ('learning_rate', 0.010597345238569207), ('max_depth', 44), ('n_estimators', 5536), ('num_leaves', 59), ('reg_alpha', 0.27257541504362476), ('reg_lambda', 0.00038430430253506674), ('subsample', 0.334370980765804)])
R2:  0.3215118024909344
Tempo total de execução: 26952.51 segundos


In [ ]:
# # Melhores parametros obtidos entre os modelos otimizados
# parametros_best_bayes = {
#     "colsample_bytree": 0.2663042735572549,
#     "learning_rate": 0.010597345238569207,
#     "max_depth": 44,
#     "n_estimators": 5536,
#     "num_leaves": 59,
#     "reg_alpha": 0.27257541504362476,
#     "reg_lambda": 0.00038430430253506674,
#     "subsample": 0.334370980765804
# }

In [24]:
# Treinar o modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.set_params(**bayes_search.best_params_)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.fit(X_train_bayes, 
                      y_train_bayes, 
                      eval_set=[(X_eval, y_eval)], 
                      eval_metric=['r2', 'rmse', 'mae'],
                      categorical_feature=categorical_features,
                      callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Warning] early_stopping_round is set=200, early_stopping_rounds=200 will be ignored. Current value: early_stopping_round=200
[LightGBM] [Info] Total Bins 29908
[LightGBM] [Info] Number of data points in the train set: 487214, number of used features: 106
[LightGBM] [Warning] early_stopping_round is set=200, early_stopping_rounds=200 will be ignored. Current value: early_stopping_round=200
[LightGBM] [Info] Start training from score 527.881246
Training until validation scores don't improve for 200 rounds
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[4993]	valid_0's rmse: 69.7303	valid_0's l1: 54.9822	valid_0's l2: 4862.32


In [25]:
# Previsões
y_pred_bayes = modelo_lgbm_bayes.predict(X_test)

In [26]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_lgbm_bayes,
                    parametros={**modelo_lgbm_bayes.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                    X_train=X_train,
                    y_train=y_train,
                    y_test=y_test,
                    y_pred=y_pred_bayes,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_bayes_censo_enem',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV Censo e ENEM 2023')

2025/06/05 06:24:01 WARNING mlflow.models.signature: Failed to infer schema for inputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. Note that MLflow doesn't validate data types during inference for AnyType. To see the full traceback, set logging level to DEBUG.
Registered model 'modelo_lgbm_bayes_censo_enem' already exists. Creating a new version of this model...
2025/06/05 06:24:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_lgbm_bayes_censo_enem, version 2


🏃 View run crawling-mule-192 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/5323bc201697419092f725769f16d1f8
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_bayes_censo_enem
Rastreamento do MLflow finalizado.


Created version '2' of model 'modelo_lgbm_bayes_censo_enem'.


In [27]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm_bayes.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_bayes, "teste")

MAE (treino): 52.9494
RMSE (treino): 67.2049
R2 (treino): 0.3698
MAE (teste): 54.7656
RMSE (teste): 69.4478
R2 (teste): 0.3235
